<a href="https://colab.research.google.com/github/niharika-sojitra/COMP-5800-Project/blob/main/CAE_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!git clone https://github.com/LeoLaugier/conditional-auto-encoder-text-to-text-transfer-transformer.git

%cd conditional-auto-encoder-text-to-text-transfer-transformer 

!pip install .

Cloning into 'conditional-auto-encoder-text-to-text-transfer-transformer'...
remote: Enumerating objects: 661, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 661 (delta 125), reused 204 (delta 124), pack-reused 456
Receiving objects: 100% (661/661), 189.09 KiB | 2.86 MiB/s, done.
Resolving deltas: 100% (400/400), done.
/content/conditional-auto-encoder-text-to-text-transfer-transformer/conditional-auto-encoder-text-to-text-transfer-transformer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/conditional-auto-encoder-text-to-text-transfer-transformer/conditional-auto-encoder-text-to-text-transfer-transformer
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes t

In [46]:
#!pip install -U tensorflow-gcs-config==2.1.3.  ==2.9.1

!pip install -U tensorflow-gcs-config==2.3

!pip install tensorflow==2.3 t5 tensorflow-text==2.3
#!pip install tensorflow t5 tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import tensorflow as tf
import os


In [48]:
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  8


In [49]:
assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

tpu_address = TF_MASTER

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [1]:
#gcloud auth application-default login
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [2]:
from google.cloud import storage


def authenticate_implicit_with_adc(project_id="your-google-cloud-project-id"):
    """
    When interacting with Google Cloud Client libraries, the library can auto-detect the
    credentials to use.

    // TODO(Developer):
    //  1. Before running this sample,
    //  set up ADC as described in https://cloud.google.com/docs/authentication/external/set-up-adc
    //  2. Replace the project variable.
    //  3. Make sure that the user account or service account that you are using
    //  has the required permissions. For this sample, you must have "storage.buckets.list".
    Args:
        project_id: The project id of your Google Cloud project.
    """

    # This snippet demonstrates how to list buckets.
    # *NOTE*: Replace the client created below with the client required for your application.
    # Note that the credentials are not specified when constructing the client.
    # Hence, the client library will look for credentials using ADC.
    storage_client = storage.Client(project=project_id)
    buckets = storage_client.list_buckets()
    print("Buckets:")
    for bucket in buckets:
        print(bucket.name)
    print("Listed all storage buckets.")

authenticate_implicit_with_adc(project_id="my-project-5800-369416")


Buckets:
buck_10
Listed all storage buckets.


In [14]:
print("Setting up GCS access...")
import tensorflow as tf
import tensorflow_gcs_config
from google.colab import auth

# Set credentials for GCS reading/writing from Colab and TPU.
TPU_TOPOLOGY = "2x2"
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  TPU_ADDRESS = tpu.get_master()
  print('Running on TPU:', TPU_ADDRESS)
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
auth.authenticate_user()
tf.config.experimental_connect_to_host(TPU_ADDRESS)
tensorflow_gcs_config.configure_gcs_from_colab_auth()

%env TPU_ADDRESS=$TPU_ADDRESS

Setting up GCS access...
Running on TPU: grpc://10.120.21.42:8470


FileNotFoundError: ignored

In [4]:
import os

base_dir = "gs://buck_10"  # @param { type: "string" }
%env BASE_DIR = $base_dir
models_dir_name = "my-project-5800" # @param { type: "string" }
%env MODELS_DIR_NAME = $models_dir_name
MODELS_DIR = os.path.join(base_dir, models_dir_name)
model_size = "large"
%env MODEL_SIZE = $model_size
model_dir_counter = 1 # @param { type: "integer" }
%env MODEL_DIR_COUNTER = $model_dir_counter
bucket = "buck_10" # @param { type: "string" }
%env BUCKET = $bucket
data_dir_name = "dat_dir" # @param { type: "string" }
%env DATA_DIR_NAME = $data_dir_name 

env: BASE_DIR=gs://buck_10
env: MODELS_DIR_NAME=my-project-5800
env: MODEL_SIZE=large
env: MODEL_DIR_COUNTER=1
env: BUCKET=buck_10
env: DATA_DIR_NAME=dat_dir


In [5]:
training_steps = 100000 # @param { type: "integer" }
%env TRAINING_STEPS = $training_steps
task = "yelp" # @param { type: "string" }
mixture = "mixture_%s" %task 
%env MIXTURE = $mixture
sequence_length_gin = os.path.join("sequence_lengths", "%s.gin" % task)
%env SEQUENCE_LENGTH_GIN = $sequence_length_gin
control_codes_gin = os.path.join("control_codes", "%s.gin" % task)
%env CONTROL_CODES_GIN = $control_codes_gin

env: TRAINING_STEPS=100000
env: MIXTURE=mixture_yelp
env: SEQUENCE_LENGTH_GIN=sequence_lengths/yelp.gin
env: CONTROL_CODES_GIN=control_codes/yelp.gin


In [6]:
!caet5 --base_dir="${BASE_DIR}" \
       --model_dir_name="${MODELS_DIR_NAME}" \
       --model_size="${MODEL_SIZE}" \
       --model_dir_counter="${MODEL_DIR_COUNTER}" \
       --tpu="${TPU_ADDRESS}" \
       --module_import=caet5.data.tasks \
       --use_model_api=True \
       --mode="finetune" \
       --bucket="${BUCKET}" \
       --data_raw_dir_name="yelp_processed" \
       --train_steps="${TRAINING_STEPS}" \
       --mixture_or_task="${MIXTURE}" \
       --base_pretrained_model_dir="gs://t5-data/pretrained_models/" \
       --gin_file="dataset.gin" \
       --gin_file="objectives/denoise.gin" \
       --gin_file="models/cae_bi.gin" \
       --gin_file="train.gin" \
       --gin_file="${SEQUENCE_LENGTH_GIN}" \
       --gin_file="${CONTROL_CODES_GIN}" \
       --gin_param="utils.tpu_mesh_shape.tpu_topology = '2x2'"

2022-11-29 01:43:01.719523: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-29 01:43:01.719594: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/local/lib/python3.7/dist-packages/tensorflow_datasets/core/utils/tf_utils.py:200: FutureWarning: Conversion of the second argument of issubdtype from `object` to `np.generic` is deprecated. In future, it will be treated as `np.object_ == np.dtype(object).type`.
  return np.issubdtype(value, super_type)
Instructions for updating:
non-resource variables are not supported in the long term
I1129 01:43:09.674908 139731389220736 file_utils.py:41] PyTorch version 1.12.1+cu113 available.
I1129 01:43:09.675199 139731389220736 file_utils.py:57] Te

In [7]:
!caet5 --base_dir="${BASE_DIR}" \
       --model_dir_name="${MODELS_DIR_NAME}" \
       --model_size="${MODEL_SIZE}" \
       --model_dir_counter="${MODEL_DIR_COUNTER}" \
       --tpu="${TPU_ADDRESS}" \
       --module_import=caet5.data.tasks \
       --use_model_api=True \
       --mode="eval" \
       --bucket="${BUCKET}" \
       --mixture_or_task="${MIXTURE}" \
       --base_pretrained_model_dir="gs://t5-data/pretrained_models/" \
       --checkpoint_mode="latest" \
       --gin_file="dataset.gin" \
       --gin_file="models/cae_bi.gin" \
       --gin_file="${SEQUENCE_LENGTH_GIN}" \
       --gin_file="${CONTROL_CODES_GIN}" \
       --gin_param="utils.tpu_mesh_shape.tpu_topology = '2x2'"

2022-11-29 01:43:20.766547: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-29 01:43:20.766604: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/local/lib/python3.7/dist-packages/tensorflow_datasets/core/utils/tf_utils.py:200: FutureWarning: Conversion of the second argument of issubdtype from `object` to `np.generic` is deprecated. In future, it will be treated as `np.object_ == np.dtype(object).type`.
  return np.issubdtype(value, super_type)
Instructions for updating:
non-resource variables are not supported in the long term
I1129 01:43:24.566224 140279690274688 file_utils.py:41] PyTorch version 1.12.1+cu113 available.
I1129 01:43:24.566413 140279690274688 file_utils.py:57] Te

In [11]:
import re
import time 

comment_attribute_pairs = [

            {"text": "these donuts have the perfect texture and taste .",
             "Destination attribute": "negative"},
            {"text": "these donuts have the perfect texture and taste .",
             "Destination attribute": "positive"},

            {"text": "good food for the price .",
             "Destination attribute": "negative"},
            {"text": "good food for the price .",
             "Destination attribute": "positive"},

            {"text": "a little dirty on the inside , but wonderful people that work there !",
                "Destination attribute": "negative"},
            {"text": "a little dirty on the inside , but wonderful people that work there !",
                "Destination attribute": "positive"},

            {"text": "i always order it when i go there and it is always awesome .",
             "Destination attribute": "negative"},
            {"text": "i always order it when i go there and it is always awesome .",
             "Destination attribute": "positive"},

            {"text": "the rest of the food there is good also and not very expensive .",
             "Destination attribute": "negative"},
            {"text": "the rest of the food there is good also and not very expensive .",
             "Destination attribute": "positive"},

            {"text": "great food , low prices , and huge quantity !",
             "Destination attribute": "negative"},
            {"text": "great food , low prices , and huge quantity !",
             "Destination attribute": "positive"},

            {"text": "so excited to have a chinese place near my office !",
                "Destination attribute": "negative"},
            {"text": "this is my go to spot for chinese food .",
                "Destination attribute": "positive"},

            {"text": "i guess i need to spell that out more clearly next time .",
             "Destination attribute": "positive"},
            {"text": "i guess i need to spell that out more clearly next time .",
             "Destination attribute": "negative"},

            {"text": "the service the last time i went was just terrible .",
             "Destination attribute": "positive"},
            {"text": "the service the last time i went was just terrible .",
             "Destination attribute": "negative"},

            {"text": "it has n't been for quite a few years .",
                "Destination attribute": "positive"},
            {"text": "it has n't been for quite a few years .",
                "Destination attribute": "negative"},

            {"text": "the food here is n't very good .",
                "Destination attribute": "positive"},
            {"text": "the food here is n't very good .",
                "Destination attribute": "negative"},

            {"text": "i am sad to see how much this place has gone downhill .",
                "Destination attribute": "positive"},
            {"text": "i am sad to see how much this place has gone downhill .",
                "Destination attribute": "negative"},

            {"text": "never again will i go back to this restaurant .",
                "Destination attribute": "positive"},
            {"text": "never again will i go back to this restaurant .",
                "Destination attribute": "negative"},

            {"text": "i would n't go here for a meal ever again .",
                "Destination attribute": "positive"},
            {"text": "i would n't go here for a meal ever again .",
                "Destination attribute": "negative"},

            {"text": "but nothing show stopping .",
                "Destination attribute": "positive"},
            {"text": "but nothing show stopping .",
                "Destination attribute": "negative"},

            {"text": "very rude , will not come back .",
                "Destination attribute": "positive"},
            {"text": "very rude , will not come back .",
                "Destination attribute": "negative"},
        ]

attribute_ids = {"negative": "0", "positive": "1"}

comments = []
for p in comment_attribute_pairs:
    comments.append(p["text"] + "|dst_attribute:" + attribute_ids[p["Destination attribute"]])

now = time.time()
# Write out the input text to text files.

model_dir = os.path.join("%s_%s" % (MODELS_DIR, str(model_dir_counter)), model_size)
predict_inputs_path = os.path.join(model_dir, "predict_inputs_%d.txt" % now)
predict_outputs_path = os.path.join(model_dir, "predict_outputs_%d.txt" % now)
# Manually apply preprocessing
with tf.io.gfile.GFile(predict_inputs_path, "w") as f:
    for c in comments:
        c = re.sub(r'\n', r"\\n", c, flags=re.S)
        f.write("%s\n" % c.lower())

predict_batch_size = len(comments)
%env PREDICT_BATCH_SIZE = $predict_batch_size
%env PREDICT_INPUTS_PATH = $predict_inputs_path
%env PREDICT_OUTPUTS_PATH = $predict_outputs_path

env: PREDICT_BATCH_SIZE=32
env: PREDICT_INPUTS_PATH=gs://buck_10/my-project-5800_1/large/predict_inputs_1669686496.txt
env: PREDICT_OUTPUTS_PATH=gs://buck_10/my-project-5800_1/large/predict_outputs_1669686496.txt


In [12]:
!caet5 --base_dir="${BASE_DIR}" \
       --model_dir_name="${MODELS_DIR_NAME}" \
       --model_size="${MODEL_SIZE}" \
       --model_dir_counter="${MODEL_DIR_COUNTER}" \
       --tpu="${TPU_ADDRESS}" \
       --module_import=caet5.data.tasks \
       --use_model_api=True \
       --mode="predict" \
       --bucket="${BUCKET}" \
       --mixture_or_task="${MIXTURE}" \
       --base_pretrained_model_dir="gs://t5-data/pretrained_models/" \
       --checkpoint_mode="latest" \
       --input_file="${PREDICT_INPUTS_PATH}" \
       --output_file="${PREDICT_OUTPUTS_PATH}" \
       --predict_batch_size="${PREDICT_BATCH_SIZE}" \
       --gin_file="dataset.gin" \
       --gin_file="models/cae_bi.gin" \
       --gin_file="infer.gin" \
       --gin_file="${SEQUENCE_LENGTH_GIN}" \
       --gin_file="${CONTROL_CODES_GIN}" \
       --gin_param="utils.tpu_mesh_shape.tpu_topology = '2x2'"

2022-11-29 01:48:22.857542: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-29 01:48:22.857600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/local/lib/python3.7/dist-packages/tensorflow_datasets/core/utils/tf_utils.py:200: FutureWarning: Conversion of the second argument of issubdtype from `object` to `np.generic` is deprecated. In future, it will be treated as `np.object_ == np.dtype(object).type`.
  return np.issubdtype(value, super_type)
Instructions for updating:
non-resource variables are not supported in the long term
I1129 01:48:26.499638 140708258637696 file_utils.py:41] PyTorch version 1.12.1+cu113 available.
I1129 01:48:26.499832 140708258637696 file_utils.py:57] Te

In [13]:
# The output filename will have the checkpoint appended so we glob to get
# the latest.
prediction_files = sorted(tf.io.gfile.glob(predict_outputs_path + "*"))
print("\nPredictions using checkpoint %s:\n" % prediction_files[-1].split("-")[-1])
with tf.io.gfile.GFile(prediction_files[-1]) as f:
    for c, g in zip(comments, f):
        if c:
            print("Initial text: " + c.split("|dst_style:")[0])
            print("Generated text: " + g)
            print()

IndexError: ignored